# Lab 07 Network Programming

Made & Presented by Bo Tang

In this lab, we will discuss the fundamental concepts of Network Programming and its application in optimizing flows and decision-making in networked systems. We will explore various network optimization problems, including shortest path, minimum cost flow, and maximum flow problems, and formulate them as linear programming (LP) models. These formulations will incorporate flow conservation, capacity constraints, and cost minimization objectives. Additionally, we will demonstrate how to use network simplex methods with Gurobi to efficiently solve network optimization problems.

In [ ]:
# install gurobipy first
! pip install gurobipy

In [ ]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

Network optimization problems arise in various fields, such as transportation, logistics, telecommunications, and supply chain management. These problems typically involve **nodes** (locations) and **edges** (routes, connections, or flows) with associated costs or capacities.


## **1. Notations**  

A network is typically represented as a **graph** $G = (N, E)$, where:  
- $N$ is the set of **nodes** (or vertices), representing locations, facilities, or entities.  
- $E$ is the set of **edges** (or arcs), representing connections, transportation routes, or communication links.

Each edge $(i, j) \in E$ has associated properties:  
- **Flow $x_{ij}$**: The amount of goods, traffic, or information moving from node $i$ to node $j$.
  - This is the **decision variable** in the optimization model, representing the amount of flow allocated to each edge.
- **Capacity $c_{ij}$**: The maximum allowable flow on edge $(i, j)$.
  - This is a **constraint parameter** that limits how much flow can pass through an edge.
- **Cost $d_{ij}$**: The cost per unit of flow on edge $(i, j)$.
   - This is often included in the **objective function** to minimize the total transportation or operational cost.  
     
In many flow-based network optimization problems, we define:  
- **Source node $s \in N$**: The node where the flow originates (e.g., a supply center or factory).  
- **Sink node $t \in N$**: The node where the flow is destined (e.g., a demand center or distribution hub).

Additionally, source and sink nodes may have:  
- **Supply $S_i$**: The amount of resources available at node $i$ (if it is a source).  
- **Demand $D_i$**: The amount of resources required at node $i$ (if it is a sink).

## **2. Constraints**

At the core of network optimization is the idea of flow management, where goods, information, or resources move from one node to another while satisfying **capacity**, **supply**, **demand** and **flow conservation** constraints.

### 2.1 Capacity Constraints

This constraint ensures that no edge carries more flow than its physical or operational limit allows. Each edge $(i, j) \in E$ has a **maximum allowable flow** $c_{ij}$, known as its **capacity**. The flow on an edge **cannot exceed its capacity**.

For all edges $(i, j) \in E$, the constraint is:  
$$
0 \leq x_{ij} \leq c_{ij}
$$  
where:  
- $x_{ij}$ is the flow on edge $(i, j)$.  
- $c_{ij}$ is the maximum capacity of edge $(i, j)$.

### 2.2 Supply and Demand Constraints
Source and sink nodes impose **supply and demand constraints** to regulate the total flow entering and leaving the network.

1. **Supply at Source Nodes**:  
   Each source node $s \in N$ has a supply amount $S_s$, meaning it must send out exactly $S_s$ units of flow:  
   $$
   \sum_{(s,j) \in E} x_{sj} = S_s, \quad \forall s \in N \text{ (sources)}
   $$  
   where $S_s > 0$ represents the amount of flow **produced** at source node $s$.  

2. **Demand at Sink Nodes**:  
   Each sink node $t \in N$ has a demand amount $D_t$, meaning it must receive exactly $D_t$ units of flow:  
   $$
   \sum_{(i,t) \in E} x_{it} = D_t, \quad \forall t \in N \text{ (sinks)}
   $$  
   where $D_t > 0$ represents the amount of flow **consumed** at sink node $t$.  

For a **balanced flow network**, the total supply must equal the total demand (**It is not constraints!!!**):  
$$
\sum_{s \in N} S_s = \sum_{t \in N} D_t
$$  
If total supply is **greater than total demand**, the network includes excess flow that must be stored or discarded. If total supply is **less than total demand**, the problem may be **infeasible** unless additional supply sources are introduced.

### 2.3 Flow Conservation Constraints

This constraint ensures that all flow that enters an intermediate node must also leave it, maintaining **flow balance**. For any **intermediate node** (i.e., a node that is neither a source nor a sink), the total **incoming flow must equal outgoing flow**.

Mathematically, for all **intermediate nodes** $i \in N \setminus \{s, t\}$:  
$$
\sum_{(j,i) \in E} x_{ji} = \sum_{(i,k) \in E} x_{ik}
$$
where:
- $\sum_{(j,i) \in E} x_{ji}$ represents the **total incoming flow** to node $i$.  
- $\sum_{(i,k) \in E} x_{ik}$ represents the **total outgoing flow** from node $i$.

### 2.4 Flow Non-Negativity Constraints

Flow must always be **non-negative**, meaning that no edge can carry negative flow:  
$$
x_{ij} \geq 0, \quad \forall (i, j) \in E
$$  
This constraint ensures that flow only moves **forward** in the directed network and does not reverse.

## **3. Network Simplex**

The Network Simplex Method is a specialized version of the Simplex Method, optimized for network flow problems. It takes advantage of the graph structure of the problem to efficiently find optimal solutions. Instead of operating on a general LP formulation, the Network Simplex method exploits the sparsity and special properties of network flow, leading to significant computational efficiency.

### 3.1 Using the Network Simplex Method in Gurobi

Gurobi **automatically recognizes** network flow problems and applies the **Network Simplex Method** when:  
- The **objective function** is a **linear cost function** (e.g., minimize transportation cost).  
- The **constraints** include:
  - **Capacity constraints** on edges.
  - **Flow conservation constraints** at intermediate nodes.
  - **Supply and demand constraints** at source and sink nodes.

If Gurobi detects this **network structure**, it will **automatically switch** to the **Network Simplex solver**, significantly improving performance.

Thus, network programming is more about formulation.

In addition,  you can explicitly set the **method parameter** in Gurobi:  
```python
m.setParam("Method", 3)  # Force Gurobi to use the Network Simplex Method
```

## **4. Common Types of Problems**

Depending on the objective, different types of network optimization problems arise, including:

- **Shortest Path Problems** (e.g., finding the fastest route from source to destination)  
- **Maximum Flow Problems** (e.g., maximizing goods transported through a supply chain)  
- **Minimum Cost Flow Problems** (e.g., distributing products at minimal transportation cost)

Below, we will explain each problem, provide a practical example, and implement a Gurobi solution.

### 4.1 Shortest Path Problem

The Shortest Path Problem aims to find the **minimum-cost route** from a source node to a destination node in a weighted network. The edge weights represent distances, time, or costs, and the goal is to minimize the total weight of the path.

#### **Why Can We Solve the Shortest Path Problem Using Linear Programming?**

Unlike most combinatorial optimization problems, the Shortest Path Problem can be formulated as a Linear Program (LP) and still yield integer solutions, meaning no need for integer programming! This is due to a fundamental property known as total unimodularity.

A matrix $A$ is totally unimodular (TUM) if every square submatrix has a determinant of 0, +1, or -1. If $A$ is TUM and the right-hand side (RHS) of the constraints is integer, then solving the LP relaxation automatically produces integer solutions.
This eliminates the need for an integer programming solver.
Total Unimodularity in the Shortest Path Problem
The constraint matrix in the flow-based LP formulation of the Shortest Path Problem is totally unimodular.
This means the LP relaxation (where $x_{ij} \in [0,1]$ instead of $x_{ij} \in {0,1}$) naturally produces integer solutions without explicitly enforcing integrality.

This is why we don’t need to solve an ILP (Integer Linear Program)—the LP relaxation alone guarantees integer solutions.

#### **Example Problem**

You are given a road network where cities are represented as **nodes**, and roads between them have **distances** (edge weights).

```
   (A)
   / \
  4   2
 /     \
(B)---(C)---(D)
   1      3
```

Your task is to find the **shortest path from City A to City D**.


In [ ]:
# define graph: nodes and edges with distances
nodes = ["A", "B", "C", "D"]
edges, distances = gp.multidict({
    ("A", "B"): 4,
    ("A", "C"): 2,
    ("B", "C"): 1,
    ("C", "D"): 3
})

In [ ]:
# create model
m = gp.Model("ShortestPath")

# decision variables: flow on each edge
x = m.addVars(edges, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="x")

# objective: Minimize total distance traveled
m.setObjective(gp.quicksum(distances[i, j] * x[i, j] for i, j in edges), GRB.MINIMIZE)

# TODO:constraints
m.addConstr

# solve model
m.optimize()

# print results
if m.status == GRB.OPTIMAL:
    print(f"\nOptimal Distance: {m.objVal}")
    print("Edges used in the shortest path:")
    for i, j in edges:
        if x[i, j].x > 0.5:
            print(f"  {i} → {j}")

### 4.2 Maximum Flow Problem

The Maximum Flow Problem aims to maximize the total amount of flow that can be sent from a source to a sink, subject to **capacity constraints** on each edge.  

This problem is widely used in **logistics, telecommunications, and network design**, where the goal is to find the most efficient way to transport goods, data, or resources through a network while respecting capacity limits.

#### **Example Problem**

You are given a road network where cities are represented as **nodes**, and roads between them have **distances** (edge weights).

```
       (S)
      /   \
    10     5
    /       \
   (A)——————(B)
    \   5   /
     5     10
      \   /
       (T)

```

In [ ]:
# define graph: nodes and edges with distances
nodes = ["S", "A", "B", "T"]
edges, capacities = gp.multidict({
    ("S", "A"): 10,
    ("S", "B"): 5,
    ("A", "B"): 5,
    ("A", "T"): 5,
    ("B", "T"): 10
})

In [ ]:
# create model
m = gp.Model("MaxFlow")

# TODO: decision variables: flow on each edge (bounded by capacity)
x = m.addVars

# TODO: objective (Hint: maximize total flow from S. Why?)
m.setObjective

# TODO: constraints
m.addConstrS

# Solve model
m.optimize()

# Print results
if m.status == GRB.OPTIMAL:
    print(f"\nMaximum Flow: {m.objVal}")
    print("Flow distribution:")
    for i, j in edges:
        print(f"  Flow on {i} → {j}: {x[i, j].x}")

### 4.3 Minimum Cost Flow Problem

The Minimum Cost Flow (MCF) Problem is an extension of the **Maximum Flow Problem**. Instead of simply maximizing the total flow from a source to a sink, the goal is to **minimize the total transportation cost** while ensuring that supply and demand constraints are satisfied.  

This problem is widely used in **logistics, supply chain management, transportation networks, and telecommunication systems**, where goods, data, or resources must be moved efficiently while minimizing transportation costs.

#### **Example Problem**  

A company has **two warehouses** supplying goods to **two retail stores**. Each warehouse has a **limited supply**, and each retail store has a **demand** for goods. There are **transportation routes** between warehouses and stores, each with a specific **cost per unit of transportation** and a **maximum transportation capacity**.

Here are the capacity and cost tables for the transportation network:

###### Capacity Table (Maximum Units that Can Be Transported)
| From → To       | Retailer 1 | Retailer 2 |
|-----------------|------------|------------|
| **Warehouse 1** | 15         | 10         |
| **Warehouse 2** | 20         | 15         |
| **Retailer 1**  | -          | 10         |

###### Cost Table (Cost per Unit of Transportation)
| From → To       | Retailer 1 | Retailer 2 |
|-----------------|------------|------------|
| **Warehouse 1** | 4          | 3          |
| **Warehouse 2** | 3          | 1          |
| **Retailer 1**  | -          | 5          |


In [ ]:
# define nodes
nodes = ["Warehouse1", "Warehouse2", "Retailer1", "Retailer2"]

# define supply and demand at each node (positive = supply, negative = demand)
supply_demand = {
    "Warehouse1": 20,  # Warehouse 1 supplies 20 units
    "Warehouse2": 30,  # Warehouse 2 supplies 30 units
    "Retailer1": -25,  # Retailer 1 requires 25 units
    "Retailer2": -25   # Retailer 2 requires 25 units
}

# define edges with (capacity, cost per unit)
edges, capacities, costs = gp.multidict({
    ("Warehouse1", "Retailer1"): [15, 4],  # (capacity, cost)
    ("Warehouse1", "Retailer2"): [10, 3],
    ("Warehouse2", "Retailer1"): [20, 3],
    ("Warehouse2", "Retailer2"): [15, 1],
    ("Retailer1", "Retailer2"): [10, 5]
})


In [ ]:
# Create Gurobi model
m = gp.Model("MinCostFlow")

# TODO: decision variables: flow on each edge
x = m.addVars

# TODO: objective
m.setObjective

# TODO: flow constraints: Inflow - Outflow
m.addConstrs

# slve model
m.optimize()

# print results
if m.status == GRB.OPTIMAL:
    print(f"\nMinimum Cost: {m.objVal}")
    print("Optimal Transportation Plan:")
    for i, j in edges:
        if x[i, j].x > 0:
            print(f"  Send {x[i, j].x:.0f} units from {i} to {j}")